## Downloading dependencies

In [ ]:
!pip install transformers accelerate bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import json

## Loading model

In [ ]:
MODEL_ID = "OpenLLM-Ro/RoMistral-7b-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

## Evaluate function (prompt to critic)

In [ ]:
def evaluate_document(text, scop_cerut=""):
    # Truncate text to first 2000 characters
    text_to_check = text[:2000]
    
    # Create prompt
    prompt = f"""
Ești un AUDITOR NOTARIAL EXPERT. Rolul tău este să verifici dimensiunile juridice ale documentului.
Trebuie să extragi și să evaluezi următoarele aspecte:
DATE DE REFERINȚĂ (Ce trebuie să existe în document)
1. Scopul Mandatului: "{scop_cerut}"
2. Mandant (persoana care dă procura)
TEXT DE VERIFICAT (primele 2000 caractere)
{text_to_check}
SARCINI DE AUDIT DETALIATE
A. VERIFICAREA STRUCTURII DOCUMENTULUI
   - Documentul are titlu potrivit tipului său?
   - Documentul are structură logică (introducere, corp, final)?
   - Există completitudine formală (lipsuri evidente, fraze incomplete, inconsistențe)?
B. VERIFICARE A LIMBAJULUI JURIDIC
   - Tonul este juridic, neutru, formal?
   - Nu apar prescurtări, structuri colocviale, contradicții sau formulări vagi?
   - Termenii juridici sunt folosiți corect (ex.: mandant, mandatar, drepturi, obligă, împuternicește)?
C. VERIFICARE A SCOPULUI ȘI A PĂRȚILOR IMPLICATE
   - Este menționat clar mandantul? (verifică și variații de nume)
   - Este menționat mandatarul sau rolul acestuia?
   - Documentul menționează explicit sau implicit acțiunea cerută: "{scop_cerut}"?
   - Acceptă sinonime juridice (ex.: “împuternicește să”, “are dreptul să”, “mandatat pentru”).
Răspunsul tău TREBUIE să fie STRICT un JSON cu următoarele câmpuri:
{{
  "structura": (int între 1-10),
  "limbaj": (int între 1-10),
  "respectare_scop": (int între 1-10),
  "observatii": "text scurt și concret"
}}
NU adăuga explicații în afara JSON-ului.
NU adăuga alt text înainte sau după JSON.
NU folosi marcaje de cod (nu ```json).
"""
    
    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    output = model.generate(**inputs, max_new_tokens=512)
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Convert to dict
    try:
        result_json = json.loads(result)
    except:
        result_json = {"error": "Model output could not be parsed as JSON", "raw_output": result}
    
    return result_json

## Evaluate documents and gather results

In [ ]:
for doc in generated_docs:
    evaluation = evaluate_document(doc["text"], scop_cerut=doc["scop"])
    result.append(evaluation)